In [1]:
import json
import pandas as pd

In [2]:
all_cards = json.load(open("../data/AllPrintings.json"))

In [ ]:
url = "https://api.scryfall.com/" + "cards/collection"
sesh = Session()
sesh.headers.update({'Content-Type' : 'application/json'})

In [ ]:
parameters = {
    'identifiers' : [{'name' : 'Nicol Bolas, God-Pharaoh (Foil Etched)',
                      'set' : '2X2'}]
}

In [32]:
my_cards = pd.read_csv("../data/my_collection.csv")

In [33]:
my_cards = my_cards[['name', 'set_code', 'treatment', 'foil', 'ID']]

In [34]:
my_cards

,name,set_code,treatment,foil,ID
0,"Nicol Bolas, God-Pharaoh",2X2,Foil Etched,1,45109693
1,Dockside Extortionist,2X2,Borderless,1,45049212
2,"Ulamog, the Infinite Gyre",2X2,Borderless,1,45049210
3,"Ulamog, the Infinite Gyre",2X2,Borderless,0,45049209
4,Grief,mh2,NaN,0,44954460
...,...,...,...,...,...
630,Arid Mesa,mh2,NaN,0,39559026
631,Dauthi Voidwalker,MH2,Retro Frame,0,39559043
632,Sylvan Anthem,mh2,NaN,1,39559056
633,Sol Talisman,mh2,NaN,1,39559084


In [45]:
just_double_masters = my_cards[my_cards.set_code == "2X2"]

In [46]:
just_double_masters

,name,set_code,treatment,foil,ID
0,"Nicol Bolas, God-Pharaoh",2X2,Foil Etched,1,45109693
1,Dockside Extortionist,2X2,Borderless,1,45049212
2,"Ulamog, the Infinite Gyre",2X2,Borderless,1,45049210
3,"Ulamog, the Infinite Gyre",2X2,Borderless,0,45049209


In [35]:
my_cards.set_code = my_cards.set_code.str.upper()
my_cards.set_code.unique()

In [38]:
double_masters = all_cards['data']['2X2']['cards']
masters_df = pd.DataFrame(double_masters)
masters_df.name

In [50]:
one_card = just_double_masters.iloc[0]

In [377]:
# now need to write out the logic for searching through the sets
# double masters will have 2 different effects, etched foil and borderless
# def where_to_look(treatment):
#     temp = treatment.lower()
    
    # borderless -> borderColor
    # etched foil -> finishes
    # extended art -> frameEffects
    # retro frame -> frameVersion
    # retro + foil etched -> frameVersion + higher collector number
    # JP Alternate Art -> ?
    # Showcase -> ?
    # Gilded Foil -> ?
    # Dungeon Module -> ?
    
    # may come back to this function but for now it's garbage
#     conversion_dict = {
#         'borderless' : ['borderColor', ['borderless']],
#         'etched foil' : ['finishes', ['etched']],
#         'extended art' : ['frameEffects', ['???']],
#         'retro frame' : ['frameVersion', ['1997']],
#         'jp alternate art' : ['???', ['???']],
#         'showcase' : ['???', ['???']],
#         'gilded foil' : ['???', ['???']],
#         'dungeon module' : ['???', ['???']]}

def lookup(name, relevant_keys, referenceDB, promo_flag = False, sta_flag = False, verbose = False):
    '''
    Documentation TBA
    Looks up the cards. Returns the results. Handles set goofiness as well.
    '''
    
    filtered = referenceDB[referenceDB.name == name]
    filtered.number = filtered.number.astype(int)
    
    if verbose:
        print("*"*4)
        print("In lookup")
        print(f"Name: {name}", 
              f"Relevant Keys: {relevant_keys}", 
              f"Promo Flag: {promo_flag}", 
              f"Strixhaven Mystical Archive Flag: {sta_flag}",
              f"DB View: \n{filtered.loc[::,['name', 'number', relevant_keys[0]]]}", sep = '\n')
    
    # extended art cards need to be handled in the very specific situation where the
    # buy-a-box promo is also that card. Buy-a-box will typically be the highest
    # numbered card in the set.
    #print(f"result: {any(filtered.isPromo)}")
    if any(filtered.isPromo) and not promo_flag:
        filtered = filtered.iloc[0:-1]
    
    
    if relevant_keys[1] == '':
        
        result = filtered.iloc[0]
    
    elif relevant_keys[1] == 'extended art':
        
        result = filtered.iloc[-1]
        
    elif relevant_keys[1] == ['showcase']:
        
        if filtered.setCode.iloc[0] != 'SNC':
            result = filtered.iloc[-1]
        else:
            result = filtered.iloc[-2]
        
    elif sta_flag:
        
        result = filtered[(filtered.number > 62)].iloc[0]
    
    elif promo_flag:
        
        result = filtered[filtered[relevant_keys[0]].isin([relevant_keys[1]])].iloc[-1]
            
    else:
        
        result = filtered[filtered[relevant_keys[0]].isin([relevant_keys[1]])].iloc[0]
    
    return result
    


def fetch_setnumber(card_info, referenceDB, verbose = False):
    '''
    Fetches the set number for a card from your binder.
    
    card_info: pd.Series A single dimentional pd.Series containing card information
    referenceDB: pd.DataFrame A dataframe containing general card information obtained via [website here]
    verbose: Bool Option to turn on debug messages.
    
    returns:
    result.number: The set number for the card in question.
    '''
    # right away we fix casing and deal with multiple treatments
    # in a single file
    treatment = card_info.treatment.lower()
    treatment = treatment.split(' (')

    
    if verbose:
        print("*"*50)
        print(card_info['name'])
        print("*"*50)
        print(f"treatment: {treatment}")
       
    # defines how we will access the information, used for
    # the diagnostics as well (see lookup function above)
    conversion_dict = {
        '' : ['borderColor', ''],
        'borderless' : ['borderColor', 'borderless'],
        'foil etched' : ['finishes', ['etched']],
        'extended art' : ['finishes', 'extended art'],
        'retro frame' : ['frameVersion', '1997'],
        'jp alternate art' : ['finishes', 'jp alt art'],
        'showcase' : ['frameEffects', ['showcase']],
        'gilded foil' : ['hasNonFoil', False],
        'dungeon module' : ['borderColor', ['???']],
        # not tested yet
        'buy-a-box' : ['hasNonFoil', False]}

    # typically the first key is the defining one, will update
    # if this is no longer a rule
    relevant_keys = conversion_dict[treatment[0]]
    
    # set up processor. Depending on how complicated this gets may move logic
    # into the lookup function and have set_code become another argument
    promo_flag = len(treatment) > 1
    sta_flag = card_info.set_code == 'STA'
             
    result = lookup(name = card_info['name'],
                    relevant_keys = relevant_keys, 
                    referenceDB = referenceDB, 
                    promo_flag = promo_flag, sta_flag = sta_flag, verbose = verbose)
    
    if verbose:
        print(result.number)
        print("-"*50)
        
    return result.number

In [277]:
my_mh2[my_mh2['name'] == 'Sol Talisman']

,name,set_code,treatment,foil,ID
622,Sol Talisman,MH2,Retro Frame (Foil Etched,1,39625494
624,Sol Talisman,MH2,,0,39559017
633,Sol Talisman,MH2,,1,39559084


In [226]:
test_treat = my_mh2[my_mh2['name'] == 'Sol Talisman'].iloc[0].treatment.lower().split(' (')

In [227]:
test_treat

['retro frame', 'foil etched']

In [116]:
keys = ['borderColor', 'borderless']
card_inf = just_double_masters.iloc[1]
print(card_inf)
filt = masters_df[masters_df.name == card_inf['name']]
print(keys[1])
filt[filt[keys[0]].isin([keys[1]])]

name         Dockside Extortionist
set_code                       2X2
treatment               Borderless
foil                             1
ID                        45049212
Name: 1, dtype: object
borderless


,artist,availability,boosterTypes,borderColor,colorIdentity,colors,convertedManaCost,finishes,foreignData,frameEffects,...,types,uuid,variations,edhrecRank,flavorText,loyalty,hasAlternativeDeckLimit,isStarter,promoTypes,isPromo
359,Douglas Shuler,[paper],NaN,borderless,[R],[R],2.0,"[nonfoil, foil]",[],[inverted],...,[Creature],110e193d-f86b-59f0-a979-fd0615c7576e,"[1104bdaa-8841-5b80-8300-0e8d0fcd33b7, 3db882c...",79.0,"The fine print is illegible scribbles, but it ...",NaN,NaN,True,[boosterfun],NaN


In [265]:
just_double_masters['set_number'] = just_double_masters.apply(lambda x: fetch_setnumber(x, masters_df), axis = 1)

**************************************************
Nicol Bolas, God-Pharaoh
**************************************************
False
['foil etched']
False
****
In lookup
Nicol Bolas, God-Pharaoh
['finishes', ['etched']]
False
False
               artist   availability boosterTypes borderColor colorIdentity  \
259  Raymond Swanland  [mtgo, paper]      [draft]       black     [B, R, U]   
529  Raymond Swanland        [paper]          NaN       black     [B, R, U]   

        colors  convertedManaCost         finishes foreignData frameEffects  \
259  [B, R, U]                7.0  [nonfoil, foil]          []          NaN   
529  [B, R, U]                7.0         [etched]          []   [inverted]   

     ...           types                                  uuid  \
259  ...  [Planeswalker]  6cac882b-57d3-5b6b-a3c9-56dcd5401f4b   
529  ...  [Planeswalker]  62278030-0f17-5cad-bdb7-9607a15bc898   

                                 variations edhrecRank flavorText loyalty  \
259  [62278030-0

/var/folders/cn/9ny0n35x1p18h680wlz6lmzm0000gn/T/ipykernel_17521/2480686054.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  just_double_masters['set_number'] = just_double_masters.apply(lambda x: fetch_setnumber(x, masters_df), axis = 1)


In [266]:
just_double_masters

,name,set_code,treatment,foil,ID,set_number
0,"Nicol Bolas, God-Pharaoh",2X2,Foil Etched,1,45109693,530
1,Dockside Extortionist,2X2,Borderless,1,45049212,360
2,"Ulamog, the Infinite Gyre",2X2,Borderless,1,45049210,337
3,"Ulamog, the Infinite Gyre",2X2,Borderless,0,45049209,337


In [23]:
all_cards['data']['MH2']['cards'][438]

{'artist': 'Volkan Baǵa',
 'availability': ['mtgo', 'paper'],
 'borderColor': 'black',
 'colorIdentity': [],
 'colors': [],
 'convertedManaCost': 0.0,
 'edhrecRank': 1659,
 'finishes': ['nonfoil', 'foil', 'etched'],
 'flavorText': 'It contains the energy of a dying star, caught at the moment of its explosion.',
 'foreignData': [{'flavorText': 'Er enthält die Energie eines sterbenden Sterns, die im Moment seiner Explosion eingefangen wurde.',
   'language': 'German',
   'multiverseId': 525674,
   'name': 'Sonnentalisman',
   'text': 'Aussetzen 3 — {1}\n{T}: Erzeuge {C}{C}.',
   'type': 'Artefakt'},
  {'flavorText': 'Contiene la energía de una estrella moribunda atrapada en plena explosión.',
   'language': 'Spanish',
   'multiverseId': 525735,
   'name': 'Talismán solar',
   'text': 'Suspender 3—{1}.\n{T}: Agrega {C}{C}.',
   'type': 'Artefacto'},
  {'flavorText': "Il contient l'énergie d'une étoile mourante, capturée au moment de son explosion.",
   'language': 'French',
   'multiverse

In [138]:
mh2_cards['frameVersion'][497]

'1997'

In [215]:
mh2_cards = pd.DataFrame(all_cards['data']['MH2']['cards'])
mh2_cards.head()

,artist,availability,boosterTypes,borderColor,colorIdentity,colors,convertedManaCost,edhrecRank,finishes,flavorText,...,isAlternative,isOnlineOnly,isRebalanced,isStarter,originalPrintings,loyalty,isReprint,watermark,promoTypes,isPromo
0,Ravenna Tran,"[mtgo, paper]",[draft],black,[W],[W],3.0,6234.0,"[nonfoil, foil]","""Into the smothering dark, Serra spoke a singl...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Eric Deschamps,"[mtgo, paper]",[draft],black,[W],[W],1.0,7928.0,"[nonfoil, foil]",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Campbell White,"[mtgo, paper]",[draft],black,[W],[W],1.0,6569.0,"[nonfoil, foil]",It doesn't purr. It hums.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Svetlin Velinov,"[mtgo, paper]",[draft],black,[W],[W],2.0,8220.0,"[nonfoil, foil]","It flickered to life, saw itself alone, and be...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,James Paick,"[mtgo, paper]",[draft],black,[W],[W],2.0,13552.0,"[nonfoil, foil]",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [321]:
my_mh2 = my_cards[my_cards.set_code == 'MH2'].copy()
my_mh2 = my_mh2.fillna("")
my_mh2

,name,set_code,treatment,foil,ID
4,Grief,MH2,,0,44954460
120,Arid Mesa,MH2,,0,44142956
198,Territorial Kavu,MH2,Retro Frame,0,43712338
199,Nettlecyst,MH2,,0,43712337
200,Dress Down,MH2,,0,43712336
...,...,...,...,...,...
629,Profane Tutor,MH2,,0,39559024
630,Arid Mesa,MH2,,0,39559026
631,Dauthi Voidwalker,MH2,Retro Frame,0,39559043
632,Sylvan Anthem,MH2,,1,39559056


In [349]:
mh2_cards[mh2_cards['name'] == 'Sanctum Prelate'].iloc[::,10:20]

,foreignData,frameVersion,hasFoil,hasNonFoil,identifiers,language,layout,legalities,manaCost,manaValue
497,[{'flavorText': '„Für dich ist der Zutritt ver...,1997,True,False,"{'cardKingdomFoilId': '247481', 'cardsphereId'...",English,normal,"{'commander': 'Legal', 'duel': 'Legal', 'legac...",{1}{W}{W},3.0


In [281]:
my_mh2[my_mh2['name'] == 'Sanctum Prelate']

,name,set_code,treatment,foil,ID,set_number


In [278]:
my_mh2['set_number'] = my_mh2.apply(lambda x: fetch_setnumber(x, mh2_cards, verbose = True), axis = 1)

**************************************************
Grief
**************************************************
['']
False
****
In lookup
Grief
['', '']
False
False
               artist   availability boosterTypes borderColor colorIdentity  \
87   Nicholas Gregory  [mtgo, paper]      [draft]       black           [B]   
316   Svetlin Velinov  [mtgo, paper]          NaN  borderless           [B]   

    colors  convertedManaCost  edhrecRank         finishes flavorText  ...  \
87     [B]                4.0      6892.0  [nonfoil, foil]        NaN  ...   
316    [B]                4.0      6892.0  [nonfoil, foil]        NaN  ...   

    isAlternative isOnlineOnly  isRebalanced  isStarter originalPrintings  \
87            NaN          NaN           NaN        NaN               NaN   
316           NaN          NaN           NaN       True               NaN   

    loyalty isReprint watermark    promoTypes  isPromo  
87      NaN       NaN       NaN           NaN      NaN  
316     NaN       Na

In [320]:
my_sta = my_cards[my_cards.set_code == 'STA'].copy()
my_sta.fillna("", inplace=True)
my_sta.head()

,name,set_code,treatment,foil,ID
425,Shock,STA,,0,39628150
426,Shock,STA,,0,39628151
427,Shock,STA,JP Alternate Art,0,39628147
433,Claim the Firstborn,STA,,0,39628139
434,Claim the Firstborn,STA,,0,39628140


In [197]:
sta_cards = pd.DataFrame(all_cards['data']['STA']['cards'])
sta_cards.head().columns

Index(['artist', 'availability', 'borderColor', 'colorIdentity', 'colors',
       'convertedManaCost', 'edhrecRank', 'finishes', 'foreignData',
       'frameVersion', 'hasFoil', 'hasNonFoil', 'identifiers', 'isReprint',
       'isStarter', 'language', 'layout', 'legalities', 'manaCost',
       'manaValue', 'name', 'number', 'printings', 'promoTypes',
       'purchaseUrls', 'rarity', 'rulings', 'securityStamp', 'setCode',
       'subtypes', 'supertypes', 'text', 'type', 'types', 'uuid', 'variations',
       'flavorText', 'keywords'],
      dtype='object')

In [195]:
sta_cards.iloc[63:73].name

63    Approach of the Second Sun
64               Day of Judgment
65                Defiant Strike
66                 Divine Gambit
67                    Ephemerate
68               Gift of Estates
69                  Gods Willing
70                    Mana Tithe
71                    Revitalize
72          Swords to Plowshares
Name: name, dtype: object

In [312]:
# used this to debug and pretty
my_sta[0:3].apply(lambda card: fetch_setnumber(card, sta_cards, verbose=True), axis = 1)

**************************************************
Shock
**************************************************
treatment: ['']
****
In lookup
Name: Shock
Relevant Keys: ['borderColor', '']
Promo Flag: False
Strixhaven Mystical Archive Flag: True
DB View: 
      name  number borderColor
43   Shock      44  borderless
106  Shock     107  borderless
44
--------------------------------------------------
**************************************************
Shock
**************************************************
treatment: ['']
****
In lookup
Name: Shock
Relevant Keys: ['borderColor', '']
Promo Flag: False
Strixhaven Mystical Archive Flag: True
DB View: 
      name  number borderColor
43   Shock      44  borderless
106  Shock     107  borderless
44
--------------------------------------------------
**************************************************
Shock
**************************************************
treatment: ['jp alternate art']
****
In lookup
Name: Shock
Relevant Keys: ['finishes', 'jp a

425     44
426     44
427    107
dtype: int64

In [314]:
my_sta.loc[::,'set_number'] = my_sta.apply(lambda card: fetch_setnumber(card, sta_cards), axis = 1)
my_sta.head(10)

,name,set_code,treatment,foil,ID,set_number
425,Shock,STA,,0,39628150,44
426,Shock,STA,,0,39628151,44
427,Shock,STA,JP Alternate Art,0,39628147,107
433,Claim the Firstborn,STA,,0,39628139,37
434,Claim the Firstborn,STA,,0,39628140,37
530,Eliminate,STA,,0,39627447,30
531,Eliminate,STA,,0,39627448,30
538,Crux of Fate,STA,,0,39627437,25
541,Sign in Blood,STA,,0,39627432,32
572,Brainstorm,STA,,0,39627400,13


In [315]:
my_cards.set_code.unique()

array(['2X2', 'MH2', 'DPA', 'USG', 'MID', 'WAR', 'RNA', 'CLB', 'MH1',
       'DKA', 'GRN', 'KLD', 'TSR', 'ZNR', 'IKO', 'AFR', 'KHM', '2XM',
       'M19', 'ORI', 'RIX', 'M20', 'MM2', 'SOI', 'OGW', 'M21', 'VOW',
       'SNC', 'STX', 'ODY', 'MM3', 'RTR', 'NEO', 'PLIST', 'PPRE', 'C15',
       'DDS', 'NCC', 'A25', 'ARB', 'ELD', 'IMA', 'NPH', 'AFC', 'MMQ',
       'EMN', 'MMA', 'SLD', 'ISD', 'ICE', 'AER', 'BAB', 'NEC', 'BNG',
       'PAFR', 'EMA', 'PD3', 'MIC', 'ZNC', 'THS', 'C17', 'PIKO', 'ZEN',
       'V10', 'STA', 'FUT', 'ZNE', 'ONS', 'AVR', 'SHM', 'V11', 'STH',
       'PSTX'], dtype=object)

In [319]:
my_snc = my_cards[my_cards.set_code == 'SNC'].copy()
my_snc.fillna('', inplace=True)
my_snc.head()

,name,set_code,treatment,foil,ID
84,Vivien on the Hunt,SNC,,0,44211869
85,Vivien on the Hunt,SNC,,0,44211870
86,Vivien on the Hunt,SNC,,0,44211871
87,Vivien on the Hunt,SNC,,0,44211872
123,Spara's Headquarters,SNC,,0,44016242


In [333]:
my_snc.treatment.unique()

array(['', 'Showcase', 'Extended Art', 'Gilded Foil'], dtype=object)

In [323]:
snc_cards = pd.DataFrame(all_cards['data']['SNC']['cards'])
snc_cards.fillna('', inplace=True)
snc_cards.head()

,artist,availability,boosterTypes,borderColor,colorIdentity,colors,convertedManaCost,edhrecRank,finishes,flavorText,...,variations,leadershipSkills,loyalty,frameEffects,isStorySpotlight,isReprint,isFullArt,isStarter,promoTypes,isPromo
0,Zack Stella,"[arena, mtgo, paper]",[draft],black,[W],[W],6.0,20382.0,"[nonfoil, foil]",Still and solemn as the statues of her kind th...,...,,,,,,,,,,
1,Aaron J. Riley,"[arena, mtgo, paper]",[draft],black,[W],[W],2.0,15931.0,"[nonfoil, foil]","""My sources say the Beamtown Bullies were spot...",...,,,,,,,,,,
2,John Stanko,"[arena, mtgo, paper]",[draft],black,[W],[W],5.0,17239.0,"[nonfoil, foil]",Don't try to cut in. They'll cut back.,...,,,,,,,,,,
3,Robin Olausson,"[arena, mtgo, paper]",[draft],black,[W],[W],1.0,8233.0,"[nonfoil, foil]","""Saving a stranger's life is an excellent recr...",...,,,,,,,,,,
4,Viko Menezes,"[arena, mtgo, paper]",[draft],black,"[G, U, W]",[W],1.0,20339.0,"[nonfoil, foil]","Regarding the relative might of pen and sword,...",...,,,,,,,,,,


In [337]:
snc_cards[snc_cards.name == "Queza, Augur of Agonies"].hasNonFoil

211     True
325     True
390    False
Name: hasNonFoil, dtype: bool

In [332]:
my_snc[my_snc['name'] == 'Queza, Augur of Agonies'].apply(lambda card: fetch_setnumber(card, snc_cards, True), axis = 1)

**************************************************
Queza, Augur of Agonies
**************************************************
treatment: ['gilded foil']
****
In lookup
Name: Queza, Augur of Agonies
Relevant Keys: ['hasNonFoil', False]
Promo Flag: False
Strixhaven Mystical Archive Flag: False
DB View: 
                        name  number  hasNonFoil
211  Queza, Augur of Agonies     212        True
325  Queza, Augur of Agonies     326        True
390  Queza, Augur of Agonies     391       False
391
--------------------------------------------------


151    391
dtype: int64

In [361]:
my_snc[my_snc['treatment'] == 'Extended Art'].apply(lambda card: fetch_setnumber(card, snc_cards, True), axis = 1)

**************************************************
Jaxis, the Troublemaker
**************************************************
treatment: ['extended art']
****
In lookup
Name: Jaxis, the Troublemaker
Relevant Keys: ['finishes', 'extended art']
Promo Flag: False
Strixhaven Mystical Archive Flag: False
DB View: 
                        name  number         finishes
111  Jaxis, the Troublemaker     112  [nonfoil, foil]
424  Jaxis, the Troublemaker     425  [nonfoil, foil]
460  Jaxis, the Troublemaker     461           [foil]
result: True
425
--------------------------------------------------


150    425
dtype: int64

In [372]:
# if it's showcase, then there is 
snc_cards[snc_cards.name == "Rocco, Cabaretti Caterer"].iloc[::,40:50]

,loyalty,frameEffects,isStorySpotlight,isReprint,isFullArt,isStarter,promoTypes,isPromo
217,,[legendary],,,,,,
331,,"[showcase, legendary]",,,,True,[boosterfun],
396,,"[showcase, legendary]",,,,True,"[gilded, boosterfun]",


In [378]:
my_snc['set_number'] = my_snc.apply(lambda card: fetch_setnumber(card, snc_cards), axis = 1)

In [380]:
my_snc.head()

,name,set_code,treatment,foil,ID,set_number
84,Vivien on the Hunt,SNC,,0,44211869,162
85,Vivien on the Hunt,SNC,,0,44211870,162
86,Vivien on the Hunt,SNC,,0,44211871,162
87,Vivien on the Hunt,SNC,,0,44211872,162
123,Spara's Headquarters,SNC,,0,44016242,257
